In [12]:
def checkRow(digit,board):
    for i in range(9):
        if digit in board[i]:
            return False
    return True

def checkCol(digit,board):
    for i in range(9):
        if digit in board[:,i]:
            return False
    return True

def checkBlock(digit,board):
    for i in range(3):
        for j in range(3):
            if digit in board[i*3:(i+1)*3,j*3:(j+1)*3]:
                return False
    return True

In [13]:
# imports

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt

import csv

In [14]:
# import trainData_1 from sudokuData/trainData_1.csv

def load_data(source):
    sudos_inputs = []
    sudos_outputs = []
    with open('sudokuData/' + source + '.csv', 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            sudos_inputs.append(row[0])
            sudos_outputs.append(row[1])
    # print(sudos_inputs[0:2])
    # print(sudos_outputs[0:2])

    # remove headers
    sudos_inputs = sudos_inputs[1:]
    sudos_outputs = sudos_outputs[1:]

    # split str into list of ints
    for i in range(len(sudos_inputs)):
        sudos_inputs[i] = [int(x) for x in sudos_inputs[i]]
        sudos_outputs[i] = [int(x) for x in sudos_outputs[i]]

    # convert to tensor
    sudos_inputs = torch.tensor(sudos_inputs)
    sudos_outputs = torch.tensor(sudos_outputs)

    # print(len(sudos_inputs))
    # print(len(sudos_outputs))

    # print(type(sudos_inputs))
    # print(sudos_inputs.shape)

    # print(type(sudos_outputs))
    # print(sudos_outputs.shape)

    print('Data Loaded! - ' + source)
    return sudos_inputs, sudos_outputs

In [15]:
train_input, train_output = load_data('trainData_1')
test_input, test_output = load_data('testData_1')

Data Loaded! - trainData_1
Data Loaded! - testData_1


In [16]:
# Custom dataset class
class CSVDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return x, y

In [17]:
# define the network class

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(81, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 81)

    def forward(self, x):
        x = torch.flatten(x, -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [18]:
# define the training function

def train(model, train_loader, criterion, optimizer, epochs):
    loss_list = []
    for epoch in range(epochs):
        for input, data in enumerate(train_loader):
            input, labels = data
            optimizer.zero_grad()
            output = model(input)
            loss = criterion(output, labels)
            loss_list.append(loss.item())
            loss.backward()
            optimizer.step()
            if epoch % 100 == 0:
                print("Epoch: " + str(epoch) + ", Loss: " + str(loss.item()))
    print("Epoch: " + str(epoch) + ", Loss: " + str(loss.item()))

    # use matplotlib to plot the loss
    plt.plot(loss_list)
                
    print("Finished Training")

In [19]:
# define the testing function

def test(model, test_loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for data, labels in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print(f"Accuracy: {correct / total}")

In [20]:
# data loader
train_dataset = CSVDataset(train_input, train_output)
test_dataset = CSVDataset(test_input, test_output)

train_loader = DataLoader(train_dataset, batch_size=250, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=250, shuffle=True)

In [21]:
# create an instance of the network
model = Net()

# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0025)

# run one forward pass on the first item in the train_loader
print(train_output[0])
model.forward(train_input[0])

tensor([6, 7, 9, 5, 1, 8, 2, 4, 3, 5, 4, 3, 7, 2, 9, 6, 1, 8, 8, 2, 1, 6, 3, 4,
        9, 5, 7, 7, 9, 4, 3, 5, 2, 1, 8, 6, 3, 5, 8, 4, 6, 1, 7, 2, 9, 2, 1, 6,
        8, 9, 7, 5, 3, 4, 4, 8, 5, 2, 7, 6, 3, 9, 1, 9, 6, 2, 1, 8, 3, 4, 7, 5,
        1, 3, 7, 9, 4, 5, 8, 6, 2])


RuntimeError: expected scalar type Long but found Float